# Introduction
Docker is a kind of virtual machine that allows to create and run instances of OS within another OS. It is a kind of shared server. Each instance runs independently and problems arising in any docker image (instance) do not propagate to other images.

## Tutorials

* [Learn about images and containers](https://docs.docker.com/engine/getstarted/step_two/)
* [Working with docker containers](https://www.digitalocean.com/community/tutorials/working-with-docker-containers)
* [Continuos Integration with Docker](https://www.digitalocean.com/community/tutorials/how-to-configure-a-continuous-integration-testing-environment-with-docker-and-docker-compose-on-ubuntu-16-04)


## Nomenclature

* **image**: is a filesystem and parameters to use at runtime. It doesn’t have state and never changes.
* **container** is a running instance of an image

A running container is **not** a real running OS. Check [here](http://phusion.github.io/baseimage-docker/#intro) for more info.

# Create a LAMP image

To create a functioning **LAMP** (Linux, Apache, ~~MySQL~~, PHP) we may save the following file and run it. It will create an image with the latest debian OS, install apache2, php, vim, git. Notice that the actual apache2 is not run by default and we execute a call to manually launch the service.


    FROM debian:latest
    RUN apt-get update
    RUN apt-get install -y apache2
    RUN apt-get install -y php5 php-pear php5-mysql
    RUN apt-get install -y vim git
    RUN apt-get install -y ssh
    RUN service apache2 restart
    EXPOSE 80

connect to vishighmem01 and 
```bash 
   sudo docker build -t "project_dashboard:dockerfile" .
```
test that the container exists
```bash
   sudo docker images | grep project_dashboard
``` 
Run the newly created project_dashboard image with port binding. The following command creates and runs a container based on the image binding the port **80** of the container to the port **1337** of the host.

```bash
sudo docker run -p 1337:80 -i -t project_dashboard:dockerfile /bin/bash
```

Due to the -i the container will run interactively and it will launch /bin/bash at start. But as you disconnect typing _exit_, you will close the main task the container was created for and the container will be **closed**. 
That is not what we want. By hitting CTRL+P and then CTRL+Q; this will leave the container running in the background.

Before getting into the problem of leaving the container running, you should be aware that if you do any modification to the content of the container, it will **not** be there when starting the container once more. In facts _the image is stateless_ and the container is the running instance of the image, therefore when it doesn't run it doesn't exist!

### Deploy something to your container 

Would you want to have some files installed in the container, those should be 
* put in at creation 
* a directory may be mapped from host to container.
* added from the running container (i.e. with git clone/pull) 

#### Copy 
To copy data to the container one can add the following command to the dockerfile:

    COPY <source> <destination>

Where _source_ is in the host and _destination_ is in the container! However, in this way the copy will be static and changes in the host filesystem will not reflect in the container. 

#### using git
I've used git to pull the project I want to the container. Because my webapp requires Apache and PHP I've run/created/attached to the container and 
    
    cd /var/www/html
    git clone https://github.com/vais-ral/Project-Dashboard.git

This will pull to the container the latest files of the Dashboard.


## Running as "daemon"

[Attach](https://docs.docker.com/engine/reference/commandline/attach/) to a running container

First get the id of the container
```bash
sudo docker ps -a
``` 
will return something afbad1472f44. 
```bash
sudo docker attach afbad1472f44 
```

If you've read the previous [link](http://phusion.github.io/baseimage-docker/#intro) you understand that running services like Apache/MySQL may be more cumbersome than expected.it may be better to use [linode images](https://www.linode.com/docs/applications/containers/how-to-install-docker-and-deploy-a-lamp-stack).

